In [6]:
from bs4 import BeautifulSoup
import requests

First parent/root soup

https://old.reddit.com/r/MechanicalKeyboards/comments/k385xo/mech_mat_giveaway/

Individual comment with replies and replies within replies soup

https://old.reddit.com/r/MechanicalKeyboards/comments/k385xo/mech_mat_giveaway/ge0pz8h/

In [21]:
def getRedditSoup(url):
    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-US, en; q=0.9",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36"
    }
    response = requests.get(url, headers = headers)

    if (response.status_code != 200):
        return BeautifulSoup("", "html.parser")

    return BeautifulSoup(response.text, "html.parser")

In [19]:
def extractCommentData(comment_tag):
    top_level_comment_object = {}
    score = comment_tag.find("span", class_="score unvoted")["title"]
    author = comment_tag.find("a", class_="author").text.strip()
    
    date_posted = comment_tag.find("time", class_="live-timestamp")
    date_posted_timestamp = date_posted["datetime"]
    date_posted_readable = date_posted["title"]
    
    date_edited = comment_tag.find("time", class_="edited-timestamp")
    date_edited_timestamp = date_edited["datetime"]\
        if date_edited != None else None
    
    num_children = comment_tag.find("a", class_="numchildren")\
        .text.strip().replace("(", "").replace(")", "").split(" ")[0]
    
    permalink = comment_tag.find("a", class_="bylink")
    
    comment_container = comment_tag\
        .find("div", class_="usertext-body may-blank-within md-container")\
        .find("div", class_="md")
    
    comment_formatted = comment_container.prettify()
    comment_raw = "".join([p.text for p in comment_container.find_all("p")]).strip()
    
    top_level_comment_object["score"] = score
    top_level_comment_object["author"] = author
#     top_level_comment_object["date_posted_timestamp"] = date_posted_timestamp
#     top_level_comment_object["date_posted_readable"] = date_posted_readable
#     top_level_comment_object["date_edited"] = date_edited
#     top_level_comment_object["num_children"] = num_children
#     top_level_comment_object["permalink"] = permalink
#     top_level_comment_object["comment_formatted"] = comment_formatted
    top_level_comment_object["comment_raw"] = comment_raw
    
    return top_level_comment_object
    
#     if (num_children == 0):
#         base case
#     else:
#         nested_soup
        

In [16]:
with open('../../samples/reddit_post_with_nested_comments.txt', 'r') as file:
    data = file.read().replace('\n', '')
    
soup = BeautifulSoup(data, "html.parser")

In [17]:
nested_comments_container = soup.find_all("div", class_=["nestedlisting"])[0]
top_level_comments = nested_comments_container.find_all("div", class_="comment", recursive=False)

top_level_comments_objects = []

In [20]:
for comment in top_level_comments:
    print(extractCommentData(comment))


{'score': '329', 'author': 'IllustriousPhysics', 'comment_raw': "Inspired by early 2000's Gundams.                     \u200b                     We love to bring high resolution art onto deskmats. The Mech Mat is no different, but for this piece, we knew we needed something different to capture the details of the main design. The illustration is drawn using vectors, which are based on mathematical formulas. This ensures the original high level of detail, regardless of how large the print area is.                     Renders :                       https://imgur.com/a/tks3kyV                       \u200b                       We are honestly speechless seeing the comments and support by the community. In spirit of Cyber Monday, we are offering a 20% discount at launch for 24 hours.  This discount will be available for both the Mech Mat and Fujin Raijin Deskmat!                       \u200b                     \u200b                       Specifications                         Timeline 

In [22]:
with open('../../samples/reddit_comment_with_replies.txt', 'r') as file:
    inside_data = file.read().replace('\n', '')
    
nested_soup = BeautifulSoup(inside_data, "html.parser")

In [23]:
container = nested_soup.find_all("div", class_=["nestedlisting"])[0]
children = container.find_all("div", class_="comment", recursive=False)

In [24]:
children[0]

<div class="thing id-t1_ge0pz8h noncollapsed comment" data-author="IllustriousPhysics" data-author-fullname="t2_q7cb79g" data-fullname="t1_ge0pz8h" data-gildings="0" data-permalink="/r/MechanicalKeyboards/comments/k385xo/mech_mat_giveaway/ge0pz8h/" data-replies="2995" data-subreddit="MechanicalKeyboards" data-subreddit-fullname="t5_2ugo7" data-subreddit-prefixed="r/MechanicalKeyboards" data-subreddit-type="public" data-type="comment" id="thing_t1_ge0pz8h" onclick="click_thing(this)">\n      <p class="parent">\n       <a name="ge0pz8h">\n       </a>\n      </p>\n      <div class="midcol unvoted">\n       <div aria-label="upvote" class="arrow up login-required access-required" data-event-action="upvote" role="button" tabindex="0">\n       </div>\n       <div aria-label="downvote" class="arrow down login-required access-required" data-event-action="downvote" role="button" tabindex="0">\n       </div>\n      </div>\n      <div class="entry unvoted">\n       <p class="tagline">\n        <a 

In [42]:
# x = children[0].find_all("div", class_="child")
# x = children[0].find_all("div", class_=["noncollapsed", "comment"], recursive=True)